In [1]:
import numpy as np
import scipy.io as io
import h5py
import deepdish as dd
import dask.array as da
import os 

### Read mat and write out as h5

In [43]:
matpath = '../../matdata/M1'
h5path = '../../H5data/M1'
fname = os.listdir(matpath)

In [44]:
# Get the file pointer
thisfile = fname[0]
print matpath+'/'+thisfile
fid = h5py.File(matpath+'/'+thisfile)
fid.keys()

../../matdata/M1/bucky090910.mat


[u'#refs#', u'Eyes', u'Spikes']

In [45]:
# Convert Eyes struct to dict
eyes_object = fid['Eyes']
trial_object = eyes_object['trial']
impath_object = eyes_object['impath']
imname_object = eyes_object['imname']
fixation_object = eyes_object['fixation']
in_sac_dur_object = eyes_object['in_sac_dur']
out_sac_dur_object = eyes_object['out_sac_dur']
fix_onset_object = eyes_object['fix_onset']
fix_offset_object = eyes_object['fix_offset']
row_object = eyes_object['row']
col_object = eyes_object['col']

eyes = {}
for c in range(trial_object.size):
    # Get all the data for this struct element
    trial = int(np.array(fid[trial_object[c,0]])[0][0])
    impath = ''.join(chr(i) for i in fid[impath_object[c,0]])
    imname = ''.join(chr(i) for i in fid[imname_object[c,0]])
    fixation = int(np.array(fid[fixation_object[c,0]])[0][0])
    in_sac_dur = np.array(fid[in_sac_dur_object[c,0]])[0][0]
    out_sac_dur = np.array(fid[out_sac_dur_object[c,0]])[0][0]
    fix_onset = np.array(fid[fix_onset_object[c,0]])[0][0]
    fix_offset = np.array(fid[fix_offset_object[c,0]])[0][0]
    row = np.array(fid[row_object[c,0]])[0][0]
    col = np.array(fid[col_object[c,0]])[0][0]
                         
    # Create a dictionary for this element 
    this_dict = {}
    this_dict['trial'] = trial
    this_dict['impath'] = impath
    this_dict['imname'] = imname
    this_dict['fixation'] = fixation
    this_dict['in_sac_dur'] = in_sac_dur
    this_dict['out_sac_dur'] = out_sac_dur
    this_dict['fix_onset'] = fix_onset
    this_dict['fix_offset'] = fix_offset
    this_dict['row'] = row
    this_dict['col'] = col
                         
    eyes[c] = this_dict
                         
print eyes[1]

{'imname': 'blackscene.jpg', 'fix_offset': 93.637, 'fixation': 2, 'out_sac_dur': 0.043000000000000003, 'col': 1093.0, 'trial': 1, 'in_sac_dur': 0.035000000000000003, 'fix_onset': 93.385999999999996, 'impath': 'stimuli/M1/scenes', 'row': 1018.0}


In [46]:
# Convert Spikes struct to dict
spikes_object = fid['Spikes']
unit_object = spikes_object['unit']
times_object = spikes_object['times']

spikes = {}
for u in range(unit_object.size):
    uu = np.array(fid[unit_object[u,0]])
    tt = np.array(fid[times_object[u,0]])
    channel = int(np.floor(uu[0][0]))
    unit = int(10*(uu[0][0] - np.floor(uu[0][0])))
    unitid = '{0:0=2d}'.format(channel)+'{0:0=2d}'.format(unit)
    times = tt[0][:]
    spikes[unitid] = times

#for u in spikes:
#    print u
#    print spikes[u][:10]

spikes['5401'][:10]

array([ 50.9509    ,  51.11846667,  51.1258    ,  51.1906    ,
        51.20433333,  51.2221    ,  51.23406667,  51.2408    ,
        51.30686667,  51.31446667])

In [47]:
# Close the file
fid.close()

In [49]:
# Put everything into a bigger dict
d = {}
d['eyes'] = eyes
d['spikes'] = spikes

# Save it using deepdish
dd.io.save(h5path+'/'+thisfile, d)

zlib
